In [43]:
import pandas as pd
import numpy as np
import datetime
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.externals import joblib
from sklearn import metrics
import matplotlib.pyplot as plt

In [44]:
#验证数据集
def get_reference_set(user_behavior_data,label_day):
    #user_behavior_data_copy = user_behavior_data
    #data_train_ui = user_behavior_data['user_id'] + user_behavior_data['item_id']
    label_day_data = user_behavior_data[user_behavior_data['daystime'] == label_day]
    #print(len(label_day_data))
    data_label_buy = label_day_data[label_day_data['behavior_type'] == 4]
    #data_label_buy_ui = data_label_buy['user_id'] + data_label_buy['item_id']
    #data_train_labeled = data_train_ui.isin(list(data_label_buy_ui))
    #print("")
    #dict = {True: 1, False: 0}
    #data_train_labeled = data_train_labeled.map(dict)
    #user_behavior_data['label'] = data_train_labeled
    #此处的去重操作能将非购买行为去除掉（按照上边的逻辑，用户对某件商品在购买前的浏览和加购物车行为也会被标记为label==1），但同时也去除了重复购买行为
    print('user_behavior_data shape before drop_duplicates function:',user_behavior_data.shape)
    data_label_buy.drop_duplicates(['user_id', 'item_id'],inplace=True)
    data_label_buy = data_label_buy[data_label_buy.item_id.isin(list(item_table.item_id))]
    return data_label_buy[['user_id', 'item_id']]

def get_label_set(user_behavior_data,label_day):
    '''
    label_day_data = user_behavior_data[user_behavior_data['daystime'] == label_day]
    label_day_data_ui = label_day_data['user_id'] + label_day_data['item_id']
    data_label_buy = label_day_data[label_day_data['behavior_type'] == 4]
    data_label_buy_ui = data_label_buy['user_id'] + data_label_buy['item_id']
    #print(data_label_buy_ui.dtypes)
    data_train_labeled = label_day_data_ui.isin(list(data_label_buy_ui))
    dict = {True: 1, False: 0}
    data_train_labeled = data_train_labeled.map(dict)
    label_day_data['label'] = data_train_labeled
    #此处的去重操作能将非购买行为去除掉（按照上边的逻辑，用户对某件商品在购买前的浏览和加购物车行为也会被标记为label==1），但同时也去除了重复购买行为
    label_day_data.drop_duplicates(['user_id', 'item_id'],inplace=True)
    print(label_day_data.shape)
    return label_day_data[['user_id', 'item_id','item_category', 'label']]
    '''
    label_day_data = user_behavior_data[user_behavior_data['daystime'] == label_day]
    data_label_buy = label_day_data[label_day_data['behavior_type'] == 4]
    print(label_day,"buy behavior before drop duplicates",data_label_buy.shape)
    data_label_buy.drop_duplicates(['user_id', 'item_id'],inplace=True)
    print(label_day,"buy behavior after drop duplicates",data_label_buy.shape)
    print(label_day,"buy behavior after drop duplicates, in sub product set",data_label_buy[data_label_buy.item_id.isin(list(item_table.item_id))].shape)
    data_label_buy['label'] = 1
    
    other_day_data = user_behavior_data[user_behavior_data['daystime'] != label_day]
    other_day_data.drop_duplicates(['user_id', 'item_id'],inplace=True)
    #print(other_day_data.head(5))
    label_data = other_day_data.merge(data_label_buy, how='left', on=['item_id','user_id']).fillna(0)
    #print(label_data.head(5))
    print(label_day,"other day data join label day buy data",label_data[label_data['label']==1].shape)
    '''
    data_label_not_buy = label_day_data[label_day_data['behavior_type'] != 4]
    print(label_day,"not buy behavior before drop duplicates",data_label_not_buy.shape)
    data_label_not_buy.drop_duplicates(['user_id', 'item_id'],inplace=True)
    print(label_day,"not buy behavior after drop duplicates",data_label_not_buy.shape)
    data_label_not_buy['label'] = 0

    label_day_data = data_label_not_buy.append(data_label_buy)
    print(label_day_data.shape)
    label_day_data.drop_duplicates(['user_id', 'item_id'], keep='last', inplace=True)
    print(label_day_data.shape)
    '''
    label_data = label_data[["item_category_x","user_id","item_id","label"]]
    #print(label_data.head(5))
    label_data.rename(columns = {"item_category_x":'item_category'},inplace=True)
    label_data.drop_duplicates(['user_id', 'item_id'],inplace=True)
    print(label_day,"buy behavior after join other day data ，then drop duplicate",label_data[label_data['label']==1].shape)
    a = label_data[label_data['label']==1]
    a = a[a.item_id.isin(list(item_table.item_id))]
    print(label_day,"buy behavior after join other day data ，then drop duplicate, then in sub product set:",a.shape)
    return label_data[['user_id', 'item_id','item_category', 'label']]


#user_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_user.csv')
#item_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_item.csv')
#user_table = user_table[user_table.item_id.isin(list(item_table.item_id))]

In [32]:
"""
reference_data_set_buy = ReferenceData
reference_data_set_buy = reference_data_set_buy[reference_data_set_buy.item_id.isin(list(item_table.item_id))]

right_res = predict_data_set_buy.merge(reference_data_set_buy,on=['item_id','user_id'])
#caculate P, R and F1  
R = right_res.shape[0]/reference_data_set_buy.shape[0]
P = right_res.shape[0]/predict_data_set_buy.shape[0]
F1 = 2*P*R/(P+R)
print('reference_data_set_buy:',reference_data_set_buy.shape[0])
print('predict_data_set:',predict_data_set.shape[0])
print('predict_data_set_buy:',predict_data_set_buy.shape[0])
print('right_res:',right_res.shape[0])
print('P:',P,'\n','R:',R,'\n','F1:',F1)
"""

"\nreference_data_set_buy = ReferenceData\nreference_data_set_buy = reference_data_set_buy[reference_data_set_buy.item_id.isin(list(item_table.item_id))]\n\nright_res = predict_data_set_buy.merge(reference_data_set_buy,on=['item_id','user_id'])\n#caculate P, R and F1  \nR = right_res.shape[0]/reference_data_set_buy.shape[0]\nP = right_res.shape[0]/predict_data_set_buy.shape[0]\nF1 = 2*P*R/(P+R)\nprint('reference_data_set_buy:',reference_data_set_buy.shape[0])\nprint('predict_data_set:',predict_data_set.shape[0])\nprint('predict_data_set_buy:',predict_data_set_buy.shape[0])\nprint('right_res:',right_res.shape[0])\nprint('P:',P,'\n','R:',R,'\n','F1:',F1)\n"

In [45]:
#use_predict_proba = False  #only suit for LR now
def predict(start_day, end_day, algorithm, use_predict_proba, validate_or_test_flag=True,prob_value=0.4):  
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    model_save_path = "../ModelFile/"
    input_filename = "../DataSet/Feature/"+"part_data_uiclf"+end_day_str+".csv"
    predict_set = pd.read_csv(input_filename)
    print("syj predict_set",predict_set.columns)
    if(validate_or_test_flag):
        predict_set_temp = predict_set.iloc[:,3:]    #这一行要去除掉
    else:
        predict_set_temp = predict_set.iloc[:,4:]
    print("syj predict_set_temp",predict_set_temp.columns)
    print("syj predict_set_temp",predict_set_temp.head(5))
    scaler = preprocessing.StandardScaler()
    scaler.fit(predict_set_temp.values)
    predict_set_temp_values = scaler.transform(predict_set_temp.values)
    predict_set_scale = pd.DataFrame(predict_set_temp_values) 
    #prdict
    train_model_predict_start_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('train model predict start time:',train_model_predict_start_time)  
    if(algorithm == 'LR'):
        #load model
        save_path_name=model_save_path+"lr_"+"train_model.m"
        print("loading model name",save_path_name)
        modle_lr = joblib.load(save_path_name) 
        if(use_predict_proba):
            res = modle_lr.predict_proba(predict_set_scale.values)
            train_model_predict_finish_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print('train model predict finish time:',train_model_predict_finish_time)      
            #join reference data set   
            res = pd.DataFrame(res)
            predict_data_set = pd.concat([predict_set,res],axis=1)
            predict_data_set.rename(columns={0:'predict_label_not_buy',1:'predict_label_buy'},inplace=True)

            predict_data_set = predict_data_set[['item_id','user_id','predict_label_buy']]
            predict_data_set_buy = predict_data_set[predict_data_set['predict_label_buy'] >0.8]
            predict_data_set_buy[['user_id','item_id','predict_label_buy']].to_csv("../DataSet/tianchi_mobile_recommendation_predict_proba.csv",index=None)
        else:   
            res = modle_lr.predict(predict_set_scale.values)
            train_model_predict_finish_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print('train model predict finish time:',train_model_predict_finish_time)      
            #join reference data set   
            res = pd.DataFrame(res)
            predict_data_set = pd.concat([predict_set,res],axis=1)
            predict_data_set.rename(columns={0:'predict_label'},inplace=True)

            predict_data_set = predict_data_set[['item_id','user_id','predict_label']]
            predict_data_set_buy = predict_data_set[predict_data_set['predict_label']==1]
        predict_data_set_buy[['user_id','item_id']].to_csv("../DataSet/tianchi_mobile_recommendation_predict.csv",index=None)
    elif(algorithm == 'RF'):
        #load model
        save_path_name=model_save_path+"RF_"+"train_model.m"
        print("loading model name",save_path_name)
        modle_rf = joblib.load(save_path_name) 
        res = modle_rf.predict(predict_set_scale.values)
        train_model_predict_finish_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print('train model predict finish time:',train_model_predict_finish_time)      
        #join reference data set   
        res = pd.DataFrame(res)
        predict_data_set = pd.concat([predict_set,res],axis=1)
        predict_data_set.rename(columns={0:'predict_label'},inplace=True)

        predict_data_set = predict_data_set[['item_id','user_id','predict_label']]
        predict_data_set_buy = predict_data_set[predict_data_set['predict_label']==1]
        #
        item_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_item.csv')
        predict_data_set_buy = predict_data_set_buy[predict_data_set_buy.item_id.isin(list(item_table.item_id))]
        #
        predict_data_set_buy[['user_id','item_id']].to_csv("../DataSet/tianchi_mobile_recommendation_predict.csv",index=None)
        print("random forest end")
    elif(algorithm == 'GBDT'):
        #load model
        save_path_name=model_save_path+"GBDT_"+"train_model.m"
        print("loading model name",save_path_name)
        modle_gbdt = joblib.load(save_path_name) 
        res = modle_gbdt.predict(predict_set_scale.values)
        res_prob = modle_gbdt.predict_proba(predict_set_scale.values)###
        res = (modle_gbdt.predict_proba(predict_set_scale.values)[:,1] > prob_value).astype(int)
       
        print("cged")
        print(res_prob)
        print(modle_gbdt.classes_)
        print("again")
        train_model_predict_finish_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print('train model predict finish time:',train_model_predict_finish_time)   
        
        #join reference data set   
        res = pd.DataFrame(res)
        
        print("before predict:",predict_set_scale.shape,"after predict",res.shape)
        
        predict_data_set = pd.concat([predict_set,res],axis=1)    #. 确认这条语句的执行结果是否正确
        predict_data_set.rename(columns={0:'predict_label'},inplace=True)

        predict_data_set = predict_data_set[['item_id','user_id','predict_label']]
        predict_data_set_buy = predict_data_set[predict_data_set['predict_label']==1]
        #
        item_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_item.csv')
        predict_data_set_buy = predict_data_set_buy[predict_data_set_buy.item_id.isin(list(item_table.item_id))]
        predict_data_set_buy[['user_id','item_id']].to_csv("../DataSet/tianchi_mobile_recommendation_predict.csv",index=None)
        print('modle_gbdt.feature_importances_:',modle_gbdt.feature_importances_)
        #
        f1_scores=[]
        precision_score=[]
        recall=[]
        prob_values = [0.1,0.2,0.3,0.35,0.38,0.4,0.42,0.45,0.5,0.6,0.7]
        if(validate_or_test_flag):
            pass
        else:
            for prob_value in [0.1,0.2,0.3,0.35,0.38,0.4,0.42,0.45,0.5,0.6,0.7]:
                predict_set_temp = (res_prob[:,1] > prob_value).astype(int)
                f1_scores.append(metrics.f1_score(predict_set[['label',]], predict_set_temp))
                precision_score.append(metrics.precision_score(predict_set[['label',]], predict_set_temp))
                recall.append(metrics.recall_score(predict_set[['label',]], predict_set_temp))
                # plot the result
            f1 = plt.figure(1)
            plt.plot(prob_values, f1_scores, label="f1_scores")
            plt.plot(prob_values, precision_score, label="precision_score")
            plt.plot(prob_values, recall, label="recall")
            plt.xlabel('NP ratio')
            plt.ylabel('f1_score')
            plt.title('prob_values of f1_score - GBDT')
            plt.legend(loc='upper right') # 标签位置
            plt.grid(True, linewidth=0.5)
            plt.show()
                
            '''f1_scores=metrics.f1_score(predict_set[['label',]], res)
            precision_score=metrics.precision_score(predict_set[['label',]], res)
            recall=metrics.recall_score(predict_set[['label',]], res)
            print('GBDT f1, accurency, recall:',f1_scores,precision_score,recall)'''
        print("GBDT end")        